In [2]:
from common_nba_api_functions import rate_limited_request
from common_db_functions import get_players_from_db
players = get_players_from_db()
player_ids = [player.player_id for player in players]
season = "2023-24"

In [3]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.orm import load_only
from sqlalchemy.exc import SQLAlchemyError
from db_config import get_session
from models import GameStats, Player
from common_nba_api_functions import rate_limited_request
session = get_session()


def fetch_and_store_player_game_stats(player_id):
    try:
        player_name = session.query(Player).filter(Player.player_id == player_id).one_or_none().name
        print(f"Fetching data for player: {player_name}")
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season, season_type_all_star="Regular Season")
        games_df = gamelog.get_data_frames()[0]
        print(games_df.head())
        games = []
        for index, game in games_df.iterrows():
            print(f"Adding new game stat for game ID: {game['Game_ID']}")
            new_game_stat = GameStats(
                player_id=db_player.id,
                game_id=game['Game_ID'],
                points=game['PTS'],
                assists=game['AST'],
                rebounds=game['REB'],
                steals=game['STL'],
                blocks=game['BLK'],
                turnovers=game['TOV'],
                fouls=game['PF'],
                minutes=game['MIN'],
                fg_made=game['FGM'],
                fg_attempts=game['FGA'],
                fg_percentage=game['FG_PCT'],
                fg3_made=game['FG3M'],
                fg3_attempts=game['FG3A'],
                fg3_percentage=game['FG3_PCT'],
                ft_made=game['FTM'],
                ft_attempts=game['FTA'],
                ft_percentage=game['FT_PCT'],
                plus_minus=game['PLUS_MINUS']
            )
            games.append(new_game_stat)
        session.add_all(games)
        session.commit()

    except SQLAlchemyError as e:
        print(f"Database error: {e}")
        session.rollback()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # It's a good practice to ensure the session is closed properly
        session.close()

for player_id in player_ids:
    rate_limited_request(2)
    fetch_and_store_player_game_stats(player_id)

    # Close session
    session.close()

Fetching data for player: Jalen Johnson
Empty DataFrame
Columns: [SEASON_ID, Player_ID, Game_ID, GAME_DATE, MATCHUP, WL, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS, PLUS_MINUS, VIDEO_AVAILABLE]
Index: []

[0 rows x 27 columns]
Fetching data for player: Trent Forrest
Empty DataFrame
Columns: [SEASON_ID, Player_ID, Game_ID, GAME_DATE, MATCHUP, WL, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS, PLUS_MINUS, VIDEO_AVAILABLE]
Index: []

[0 rows x 27 columns]
Fetching data for player: Seth Lundy
Empty DataFrame
Columns: [SEASON_ID, Player_ID, Game_ID, GAME_DATE, MATCHUP, WL, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS, PLUS_MINUS, VIDEO_AVAILABLE]
Index: []

[0 rows x 27 columns]
Fetching data for player: Kobe Bufkin
Empty DataFrame
Columns: [SEASON_ID, Player_ID, Game_ID, GAME_DATE, MATCHUP, WL, MIN, FGM, F

KeyboardInterrupt: 